Create an API request from AeroDataBox for flights forcast for our desired cities. 

In [ ]:
import pandas as pd
from dateutil import parser
import requests
from datetime import datetime, timedelta
from pytz import timezone
import sqlalchemy
cities_icao = ["EDDB", "EDDH", "EDDM"]
flight = []
berlin_timezone = timezone("Europe/Berlin")
today = datetime.now(berlin_timezone).date()
tomorrow = (today + timedelta(days=1))

for city in cities_icao:
    times = [["00:00", "11:59"], ["12:00", "23:59"]]
    for time in times:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{city}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
        querystring = {"direction": "Both", "withCancelled": "true", "withCodeshared": "true", "withCargo": "false", "withPrivate": "true", "withLocation": "false"}
        headers = {
            "X-RapidAPI-Key": "key",
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }
        response = requests.get(url, headers=headers, params=querystring)
        if response.status_code == 200:
            flight_tem = response.json()
            flights_all = pd.json_normalize(flight_tem["arrivals"])
            flights_all["city_arrival"] = city
            flight.append(flights_all)
        else:
            print(f"Failed to fetch data for {city}, response.status_code")

flights = pd.concat(flight, ignore_index=True)
flights=pd.DataFrame(flights, columns=["number", "status", "movement.airport.icao", "movement.airport.name", "movement.scheduledTime.local", "city_arrival"])
flights_fetch=flights.rename(columns={"movement.airport.icao" : "departure_airport_icao", "movement.airport.name" : "departure_airport", "movement.scheduledTime.local" : "arrival_time", "city_arrival" : "arrival_airport_icao"})
flights_fetch
flights_fetch['arrival_time'] = flights_fetch['arrival_time'].apply(lambda x: parser.parse(x).replace(tzinfo=None))
flights_fetch

def flights_to_sql():
    schema = ""
    host = ""
    user = ""
    password = ""
    port = 3306 
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    flights_fetch.to_sql("flights", if_exists="replace", con=connection_string, index=False)

flights_to_sql()
flights_fetch